In [1]:
import glob # para mexer com vários arquivos varrendo todos com mesma extensão e transformando num grande data set
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt # mais usadas essa e a de baixo
import seaborn as sns
import warnings
import requests
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

warnings.filterwarnings('ignore')
%matplotlib inline
matplotlib.style.use('default')
plt.rcParams['figure.figsize'] = [6, 4]

#Pandas Setting
pd.options.display.float_format = '{:,.8f}'.format

In [20]:
# lista com o caminho + nome dos arquivos .csv (apenas caminho + nome.csv). não tem nenhuma informação extra

file_list = [f for f in glob.glob("/home/andre/Documents/2020.1/TCC/dados_durante_partida/dados_premier_league.csv")]

In [21]:
for i in file_list:
    print(i)

/home/andre/Documents/2020.1/TCC/dados_durante_partida/dados_premier_league.csv


In [22]:
# obs: o "encoding = 'unicode_escape'" é usado para não dá erro que daria em alguns casos qnd 
# se lê como dataframe e há algum problema de codificação, então isso evita o erro

df = pd.read_csv(file_list[0],encoding = 'unicode_escape') # df é um dataframe apenas para o arquivo arquivo 21 do file_list
display(df.shape) # mostra as linhas e colunas. Se usasse num for podia mostrar linhas e colunas de cada arquivo
df.head()

(1515, 127)

,pais,campeonato,temporada,time_casa,time_visitante,golscasaht,golsvisitanteht,golscasaft,golsvisitanteft,resultadoht,...,desarmescasaht,desarmesvisitanteht,interceptcasaft,interceptvisitanteft,interceptcasaht,interceptvisitanteht,distanciascasaft,distanciastvisitanteft,distanciascasaht,distanciastvisitanteht
0,England,Premier League,15/16,Manchester United,Tottenham,1,0,1,0,1x0,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,England,Premier League,15/16,Bournemouth,Aston Villa,0,0,0,1,0x0,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,England,Premier League,15/16,Leicester City,Sunderland,3,0,4,2,3x0,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,England,Premier League,15/16,Norwich City,Crystal Palace,0,1,1,3,0x1,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,England,Premier League,15/16,Everton,Watford,0,1,2,2,0x1,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [23]:
# criando um dataframe para todos os arquivos do file_list e não apenas para um como feito anteriormente
# ou seja é uma lista de dataframes

df_list = [pd.read_csv(f,encoding = 'unicode_escape') for f in file_list]

In [24]:
df_list[0].columns # aprensenta as colunas do dataframe 0

Index(['pais', 'campeonato', 'temporada', 'time_casa', 'time_visitante',
       'golscasaht', 'golsvisitanteht', 'golscasaft', 'golsvisitanteft',
       'resultadoht',
       ...
       'desarmescasaht', 'desarmesvisitanteht', 'interceptcasaft',
       'interceptvisitanteft', 'interceptcasaht', 'interceptvisitanteht',
       'distanciascasaft', 'distanciastvisitanteft', 'distanciascasaht',
       'distanciastvisitanteht'],
      dtype='object', length=127)

In [25]:
new_df_list = []
for i in range(len(df_list)):
    new_df_list.append(df_list[i])

In [26]:
final_dataset = pd.concat(new_df_list, sort=False).reset_index(drop=True) # o reset_index é para remover a coluna de índice
final_dataset.shape

(1515, 127)

In [27]:
display(final_dataset.head())

,pais,campeonato,temporada,time_casa,time_visitante,golscasaht,golsvisitanteht,golscasaft,golsvisitanteft,resultadoht,...,desarmescasaht,desarmesvisitanteht,interceptcasaft,interceptvisitanteft,interceptcasaht,interceptvisitanteht,distanciascasaft,distanciastvisitanteft,distanciascasaht,distanciastvisitanteht
0,England,Premier League,15/16,Manchester United,Tottenham,1,0,1,0,1x0,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,England,Premier League,15/16,Bournemouth,Aston Villa,0,0,0,1,0x0,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,England,Premier League,15/16,Leicester City,Sunderland,3,0,4,2,3x0,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,England,Premier League,15/16,Norwich City,Crystal Palace,0,1,1,3,0x1,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,England,Premier League,15/16,Everton,Watford,0,1,2,2,0x1,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [28]:
display(final_dataset.tail())

,pais,campeonato,temporada,time_casa,time_visitante,golscasaht,golsvisitanteht,golscasaft,golsvisitanteft,resultadoht,...,desarmescasaht,desarmesvisitanteht,interceptcasaft,interceptvisitanteft,interceptcasaht,interceptvisitanteht,distanciascasaft,distanciastvisitanteft,distanciascasaht,distanciastvisitanteht
1510,England,Premier League,18/19,Crystal Palace,Bournemouth,3,1,5,3,3x1,...,11.00000000,8.00000000,13.00000000,13.00000000,2.00000000,8.00000000,23.00000000,19.00000000,11.00000000,13.00000000
1511,England,Premier League,18/19,Southampton,Huddersfield Town,1,0,1,1,1x0,...,6.00000000,18.00000000,9.00000000,10.00000000,6.00000000,6.00000000,16.00000000,26.00000000,6.00000000,10.00000000
1512,England,Premier League,18/19,Watford,West Ham United,0,2,1,4,0x2,...,12.00000000,5.00000000,4.00000000,15.00000000,2.00000000,11.00000000,12.00000000,34.00000000,5.00000000,17.00000000
1513,England,Premier League,18/19,Liverpool,Wolverhampton,1,0,2,0,1x0,...,9.00000000,7.00000000,9.00000000,9.00000000,6.00000000,7.00000000,15.00000000,20.00000000,5.00000000,11.00000000
1514,England,Premier League,18/19,Tottenham,Everton,1,0,2,2,1x0,...,12.00000000,7.00000000,9.00000000,11.00000000,3.00000000,6.00000000,17.00000000,17.00000000,6.00000000,3.00000000


In [29]:
final_dataset.info() # podemos ver que temos informações nulas
# pois no total temos colunas (que aqui estão em forma de linha) que não chega no total de índice que é
# 33248

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1515 entries, 0 to 1514
Columns: 127 entries, pais to distanciastvisitanteht
dtypes: float64(46), int64(56), object(25)
memory usage: 1.3+ MB


In [14]:
final_dataset = final_dataset.drop(columns=['BWH','BWD','BWA','IWH','IWD','IWA','LBH','LBD','LBA','BbAH','BbAHh','BbMxAHH','BbAvAHH','BbMxAHA',
'BbAvAHA','SJH','SJD','SJA','GBH','GBD','GBA','BSH','BSD','BSA'])

In [30]:
final_dataset.info() # podemos ver que temos informações nulas
# pois no total temos colunas (que aqui estão em forma de linha) que não chega no total de índice que é
# 33248

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1515 entries, 0 to 1514
Columns: 127 entries, pais to distanciastvisitanteht
dtypes: float64(46), int64(56), object(25)
memory usage: 1.3+ MB


In [31]:
# aqui removemos as colunas nulas 

final_dataset.dropna(inplace=True) 
final_dataset.shape

(0, 127)

In [32]:
# resetando o índice de novo já que dropou para ir de 0 até o final
# sem ficar pulando nenhum valor

final_dataset = final_dataset.reset_index(drop=True)

In [33]:
final_dataset.info()

# agora podemos perceber que o total de índice é 33078 e todas as colunas do datasetfinal (HomeTeam, ..., Div)
# tem também 33078 no total

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Columns: 127 entries, pais to distanciastvisitanteht
dtypes: float64(46), int64(56), object(25)
memory usage: 38.0+ bytes


In [28]:
data = final_dataset.loc[0,'Date'].split('/')[::-1]
ano = data[0]
data[0] = ano
'-'.join(data)

'2017-08-11'

In [29]:
for key, row in final_dataset.iterrows():
    data = row['Date'].split('/')[::-1]
    if len(data[0]) == 2:
        ano = '20' + data[0]
    data[0] = ano
    final_dataset.loc[key,'Date'] = '-'.join(data)

In [34]:
final_dataset.to_csv('dados_durante_partida_2.csv', sep=';')